### [k진수에서 소수 개수 구하기](https://programmers.co.kr/learn/courses/30/lessons/92335)


In [ ]:
def solution(n, k):
    answer = 0
    def ten_to_n(ten,n):
        tmp = ""
        while ten > 0:
            ten, d = divmod(ten,n)
            tmp+=str(d)
        return tmp[::-1]

    def isprime(n):
        for i in range(2,int(n**0.5)+1):
            if n % i == 0:
                return False
        return True
    n_to_k = str(ten_to_n(n, k))
    n_to_k = n_to_k.split("0")
    for i in n_to_k:
        if not i:
            continue
        if int(i) >1 and isprime(int(i)):
            answer +=1
    return answer

### [주차 요금 계산](https://programmers.co.kr/learn/courses/30/lessons/92341)


In [82]:
import math
def solution(fees, records):
    answer = []
    def change_time(time:str):
        time = time.split(":")
        return int(time[0])*60+int(time[1])
    
    cars ={}
    for record in records:
        time, car_num, io = record.split()
        if car_num not in cars:
            cars[car_num] = {"IN":[], "OUT":[]}
        cars[car_num][io].append(change_time(time))
    
    for car in sorted(cars.keys()):
        if len(cars[car]["IN"]) > len(cars[car]["OUT"]):
            cars[car]["OUT"].append(change_time("23:59"))
        fee = 0
        time = 0
        for i in range(len(cars[car]["IN"])):
            time += cars[car]["OUT"][i]-cars[car]["IN"][i]
        time-=fees[0]
        if time <=0:
            fee+= fees[1]
        else:
            fee+=fees[1]+ math.ceil(time / fees[2])*fees[3]
        answer.append(fee)
    # print(cars)
    
    return answer

In [83]:
solution([180, 5000, 10, 600],["05:34 5961 IN", "06:00 0000 IN", "06:34 0000 OUT", "07:59 5961 OUT", "07:59 0148 IN", "18:59 0000 IN", "19:09 0148 OUT", "22:59 5961 IN", "23:00 5961 OUT"])

{'5961': {'IN': [334, 1379], 'OUT': [479, 1380]}, '0000': {'IN': [360, 1139], 'OUT': [394, 1439]}, '0148': {'IN': [479], 'OUT': [1149]}}


[14600, 34400, 5000]

### [양궁대회](https://programmers.co.kr/learn/courses/30/lessons/92342)

In [125]:
from collections import defaultdict


def solution(n, info):
    answer = defaultdict(list)
    answer[-1].append([-1])

    def compare_score(a_arrrows, r_arrows):
        r_score = a_score = 0
        for idx, (a, r) in enumerate(zip(a_arrrows, r_arrows)):
            if a < r:
                r_score += 10 - idx
            else:
                if a == 0:
                    continue
                a_score += 10 - idx

        return (r_score - a_score, r_arrows) if r_score > a_score else (-1, [-1])

    def dfs(left_arrow: int, info_idx: int, r_arrow: list):
        if left_arrow <= 0:
            if len(r_arrow) < len(info):
                for _ in range(len(info) - len(r_arrow)):
                    r_arrow.append(0)
            score, r_list = compare_score(info, r_arrow)  # score, r_list
            if score == -1:
                return
            answer[score].append(r_list)
            return
        elif info_idx == 11:
            r_arrow[-1]+=left_arrow
            score, r_list = compare_score(info, r_arrow)  # score, r_list
            if score == -1:
                return
            answer[score].append(r_list)
            return
        # 1점차 승리
        if left_arrow > info[info_idx]:
            dfs(
                left_arrow - (info[info_idx] + 1),
                info_idx + 1,
                r_arrow + [info[info_idx] + 1],
            )
        # 안 쏘고 그냥 넘어가기
        dfs(left_arrow, info_idx + 1, r_arrow + [0])

    dfs(n, 0, [])
    print(answer[max(answer.keys())])
    return sorted(answer[max(answer.keys())], key=lambda x: x[::-1], reverse= True)[0]

### [양과 늑대](https://programmers.co.kr/learn/courses/30/lessons/92343)

In [137]:
from collections import defaultdict


def solution(info, edges):
    answer = 0

    graph = defaultdict(list)
    for a, b in edges:
        graph[a].append(b)
    max_cnt = -1

    def dfs(sheep: int, wolf: int, node: int, basket: set):
        nonlocal max_cnt
        if sheep == wolf:
            max_cnt = max(max_cnt, sheep)
            return
        basket |= set(graph[node])
        for n in basket:
            if info[n]:
                dfs(sheep, wolf + 1, n, basket - set([n]))
            else:
                dfs(sheep + 1, wolf, n, basket - set([n]))
        max_cnt = max(max_cnt, sheep)

    dfs(1, 0, 0, set())

    print(max_cnt)
    return answer

### [파괴되지 않은 건물](https://programmers.co.kr/learn/courses/30/lessons/92344)

효율성 체크 통과하지 못했다..

풀이를 보니 <u>**누적합**</u> 을 사용하면 된다고 한다.

In [169]:

def solution(board, skill):
    answer = 0
    acc_board =[[0 for _ in range(len(board[0])+1)] for _ in range(len(board)+1)]
    for att_hl, r1, c1, r2, c2, degree in skill:
        if att_hl == 1:
            degree = -degree 
        acc_board[r1][c1] += degree
        acc_board[r2+1][c1] -= degree
        acc_board[r1][c2+1] -= degree
        acc_board[r2+1][c2+1] += degree
    
    def do_acc(ac_board):
        for i in range(len(ac_board)):
            for j in range(len(ac_board[0])):
                if j == 0:
                    continue
                ac_board[i][j] += ac_board[i][j-1]
        for i in range(len(ac_board[0])):
            for j in range(len(ac_board)):
                if j == 0:
                    continue
                ac_board[j][i] += ac_board[j-1][i]
        return ac_board
    
    acc_board = do_acc(acc_board)

    for i in range(len(board)):
        for j in range(len(board[0])):
            board[i][j] += acc_board[i][j]
            if board[i][j] >0:
                answer +=1

    return answer

In [170]:
solution([[5,5,5,5,5],[5,5,5,5,5],[5,5,5,5,5],[5,5,5,5,5]]	,[[1,0,0,3,4,4],[1,2,0,2,3,2],[2,1,0,3,1,2],[1,0,1,3,3,1]]	)

10